In [ ]:
import mlflow
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from your_package.data.read import file_to_df
from your_package.data.transform import drop_pattern
from your_package.data.write import df_to_file
from your_package.data.features import pattern_std
from your_package.model.train_model import train_model
from your_package.utils import pipeline_to_name
from your_package.model.evaluators import evaluate_binary_class

In [ ]:
df = file_to_df("../data/pdm/raw/PredictiveManteinanceEngineTraining.csv")
df = drop_pattern(df, "setting.*")
df["s_std"] = pattern_std(df, "s[0-9][0-9]?")
df_to_file(df, "../data/processed/PredictiveManteinanceEngineTraining.csv")

print("complete")

model = Pipeline([("scaler", StandardScaler()), ("classifier", LogisticRegression())])
with mlflow.start_run(run_name=pipeline_to_name(model)) as run:
    model = train_model(
        model, df.drop(["label1", "label2", "RUL"], axis=1), df["label1"]
    )
    evaluate_binary_class(
        model, X=df.drop(["label1", "label2", "RUL"], axis=1), y=df["label1"]
    )
mlflow.end_run()